In [287]:
import pandas as pd
import numpy as np
import time

pd.set_option('max.columns', 131)
%matplotlib inline
%pylab inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, vstack
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

from sklearn.metrics import roc_auc_score, average_precision_score
from skopt import forest_minimize

from sklearn.pipeline import make_pipeline

Populating the interactive namespace from numpy and matplotlib


c:\users\diego.mayer\documents\developer\projects\movie_recommendation\venv\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['vstack', 'hstack']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [168]:
df = pd.read_csv('labels_curso - to_label_2.csv', index_col=0
                ).dropna(subset=['y'])

In [169]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

## 1. Limpeza da data

In [170]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Limpeza de Views

In [171]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [172]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [173]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [174]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = (df_limpo['date'] >= "2019-04-01")

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((555, 2), (609, 2), (555,), (609,))

In [175]:
title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

## Vectorizer

In [176]:
title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,3))

In [177]:
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [178]:
title_bow_train.shape

(555, 1144)

### Hstack

In [179]:
Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [180]:
Xtrain_wtitle.shape # included two more columns

(555, 1146)

# Random Forest

In [283]:
mdl_rf = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight="balanced", n_jobs=6)
mdl_rf.fit(Xtrain_wtitle, ytrain)

p_rf = mdl_rf.predict_proba(Xval_wtitle)[:,1]

average_precision_score(yval, p_rf), roc_auc_score(yval, p_rf)

(0.2284201947891743, 0.6926785398360559)

# LGBM

In [190]:
mdl = LGBMClassifier(random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

p_lg = mdl.predict_proba(Xval_wtitle)[:, 1]

average_precision_score(yval, p_lg), roc_auc_score(yval, p_lg)

c:\users\diego.mayer\documents\developer\projects\movie_recommendation\venv\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


(0.17930936519845178, 0.6177389868490889)

# Bayesian Optimazation
    LGBM

In [279]:
def tune_lgbm(params, tune=True):
    
    # Parameters
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    min_df = params[6]
    ngram_range = params[7]
    
    # Vectorization
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=(1, params[7]))
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)
    
    # hstack to concat
    Xtrain_wtitle = hstack([Xtrain, title_bow_train])
    Xval_wtitle = hstack([Xval, title_bow_val])
    
    # Model
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2, max_depth=max_depth, 
                        min_child_samples=min_child_samples, subsample=subsample,
                        colsample_bytree=colsample_bytree, bagging_freq=1, 
                        n_estimators=n_estimators, random_state=0,
                        class_weight='balanced', n_jobs=6)
    mdl.fit(Xtrain_wtitle, ytrain)
    
    # Probability prediction
    p_lgbm = mdl.predict_proba(Xval_wtitle)[:, 1]
    
    # if tune is True than execute for tunning else, it's validation or production
    if tune: 
        return -average_precision_score(yval, p_lgbm)
    else:
        print(average_precision_score(yval, p_lgbm), roc_auc_score(yval, p_lgbm)) 
        return p_lgbm, mdl, title_vec

## Space and Minimize for Logistic Regression

In [184]:
# space is a range or dimension that it's possible combine for get the best combination for best score
space = [
    (1e-3, 1e-1, 'log-uniform'), # lr
    (1, 11), # max_depth
    (1, 20), # min_child_samples
    (0.05, 1.), # subsample
    (0.05, 1.), # colsample_bytree
    (100, 1000), # n_estimators
    (1, 5), # min_df
    (1, 5), # ngram_range
]
# Function that you can get the best parameters
res = forest_minimize(func=tune_lgbm, dimensions=space, random_state=160745, n_random_starts=20, n_calls=100, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.1232
Function value obtained: -0.1185
Current minimum: -0.1185
Iteration No: 2 started. Evaluating function at random point.


c:\users\diego.mayer\documents\developer\projects\movie_recommendation\venv\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
c:\users\diego.mayer\documents\developer\projects\movie_recommendation\venv\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.2740
Function value obtained: -0.1203
Current minimum: -0.1203
Iteration No: 3 started. Evaluating function at random point.


c:\users\diego.mayer\documents\developer\projects\movie_recommendation\venv\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.2065
Function value obtained: -0.1175
Current minimum: -0.1203
Iteration No: 4 started. Evaluating function at random point.


...

Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 0.6146
Function value obtained: -0.1698
Current minimum: -0.2181
Iteration No: 99 started. Searching for the next optimal point.


c:\users\diego.mayer\documents\developer\projects\movie_recommendation\venv\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 0.5952
Function value obtained: -0.2282
Current minimum: -0.2282
Iteration No: 100 started. Searching for the next optimal point.


c:\users\diego.mayer\documents\developer\projects\movie_recommendation\venv\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 0.5649
Function value obtained: -0.2024
Current minimum: -0.2282


In [280]:
p_lgbm, mdl_lgbm, title_vec = tune_lgbm(res.x, tune=False)

0.22817943310199457 0.6058854750250634


c:\users\diego.mayer\documents\developer\projects\movie_recommendation\venv\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


    0.19202825084877856 0.6055021525033909 - 1,2 n_gram
    0.22817943310199457 0.6058854750250634 - 1,3
    0.230251348614118   0.6037624579819543 - 1,4
    0.22778106565523465 0.6025830040691159 - 1,5

# Logistic Regression

In [194]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

# in this case was warst than MaxAbsScaler
#scaler = StandardScaler()
#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())
scaler = MaxAbsScaler()
Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2) # with max we can put all columns easier than standardscaler
Xval_wtitle2 = scaler.transform(Xval_wtitle2)


mdl = LogisticRegression(C=0.3, random_state=0, n_jobs=6, penalty='l2')
mdl.fit(Xtrain_wtitle2, ytrain)

p_lr = mdl.predict_proba(Xval_wtitle2)[:, 1]

average_precision_score(yval, p_lr), roc_auc_score(yval, p_lr)

(0.22125869093931344, 0.6827858701421242)

### Metrics Logistic Regression
    ap -> 0.20739038717036196 roc -> 0.662027481276169 -> StandardScaler()
    ap -> 0.2052824213278625  roc -> 0.686530636315386 -> MaxAbsScaler()
    ap -> 0.2150545436183453  roc -> 0.687031904228342 -> MaxAbsScaler() params(c=0.5)
    ap -> 0.2212586909393134  roc -> 0.682785870142124 -> MaxAbsScaler() params(c=0.3)

# Pipeline

In [195]:
lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(C=0.3, penalty='l2', n_jobs=6, random_state=0))
lr_pipeline.fit(Xtrain_wtitle2, ytrain)

p_lr = lr_pipeline.predict_proba(Xval_wtitle2)[:, 1]

average_precision_score(yval, p_lr), roc_auc_score(yval, p_lr)

(0.22125869093931344, 0.6827858701421242)

# Metrics Report

    Random Forest
        ap -> 0.19423876065453383 roc -> 0.677802677360382 -> min_df=2, ngram_range=(1,1)
        ap -> 0.20385285274265763 roc -> 0.680721825794657 -> min_df=1, ngram_range=(1,2)
        ap -> 0.21799797498952303 roc -> 0.683242908533349 -> min_df=2, ngram_range=(1,2)
        ap -> 0.2284201947891743  roc -> 0.692678539836055 -> min_df=2, ngram_range=(1,3)

    LightGBM
        ap -> 0.17930936519845178 roc -> 0.617738986849088o -> no paramas
        ap -> 0.22817943310199457 roc -> 0.6058854750250634 -> [0.06584043384269299, 4, 20, 
                                                                0.8668305344607656, 0.07301246723740631,
                                                                822, 5, 3]

    LogisticRegression
        ap -> 0.21516167484835547 roc -> 0.6830807336203338 -> no params
        ap -> 0.2212586909393134  roc -> 0.682785870142124  -> MaxAbsScaler() params(c=0.3)

# Ensemble

    LG = (0.22125869093931344, 0.6827858701421242)
    lgbm = (0.22817943310199457, 0.6058854750250634)
    RF - (0.2284201947891743, 0.6926785398360559)

In [260]:
p = (p_rf + p_lgbm + p_lr)
average_precision_score(yval, p), roc_auc_score(yval, p)

In [284]:
pd.DataFrame({'RF':p_rf, 'LR':p_lr, 'LGBM':p_lgbm}).corr()
# This case is important analyse that how less correlation between them it will be better for ensemble
# because it's diferent solution but the some performance, to get more simple way solution because the deploy
# let's doing ensemble just with RF and LGBM

,RF,LR,LGBM
RF,1.000000,0.812706,0.377971
LR,0.812706,1.000000,0.481204
LGBM,0.377971,0.481204,1.000000


In [273]:
p = p_rf*0.1 + p_lgbm*0.9
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.2398470587807427, 0.6143480568496785)

    ngram = 1,3
    lr + rf + lgbm -> (0.2583693577701163,  0.6576045291030252)
         rf + lgbm -> (0.24187724466569893, 0.6233708792828919) 0.2 + 0.8
         rf + lgbm -> (0.25879704689816807, 0.6321283245857168) 0.3 + 0.7
         rf + lgbm -> (0.25381192004628994, 0.6567199386683965) 0.4 + 0.6
         rf + lgbm -> (0.2687715019710023,  0.6479624933655718) 0.5 + 0.5  <--- best
         rf + lgbm -> (0.25381192004628994, 0.6567199386683965) 0.6 + 0.4
         rf + lgbm -> (0.24708998095614865, 0.6670106740579111) 0.7 + 0.3
         rf + lgbm -> (0.236449556145549,   0.6750899333608539) 0.8 + 0.2
         rf + lgbm -> (0.23366883200760735, 0.6845550510113818) 0.9 + 0.1

# Save Models

In [285]:
import joblib as jb

In [286]:
jb.dump(mdl_lgbm, "lgbm_20200611.pkl.z")
jb.dump(mdl_rf, "random_forest_20200611.pkl.z")
jb.dump(title_vec, "title_vectorizer_20200611.pkl.z")

['title_vectorizer_20200611.pkl.z']